In [1]:
import os

In [2]:
%pwd

'd:\\python_project\\MLOPS_data_science_project\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\python_project\\MLOPS_data_science_project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.data_science.constants import *
from src.data_science.utils.common import read_yaml, create_directories

In [21]:
class ConfigurationManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config=read_yaml(config_path)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )

        return data_ingestion_config

In [16]:
import os
import urllib.request as request
from src.data_science import logger
import zipfile

In [17]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    
    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [26]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-21 16:40:21,113: INFO: common: yaml file: config\config.yaml loaded successfully ]
[2024-12-21 16:40:21,115: INFO: common: yaml file: params.yaml loaded successfully ]
[2024-12-21 16:40:21,116: INFO: common: yaml file: schema.yaml loaded successfully ]
[2024-12-21 16:40:21,117: INFO: common: created directory at: artifacts ]
[2024-12-21 16:40:21,119: INFO: common: created directory at: artifacts/data_ingestion ]


[2024-12-21 16:40:22,076: INFO: 2514658180: artifacts/data_ingestion/data.zip download! with following info: 
Content-Type: text/html; charset=utf-8
X-Robots-Tag: noindex, nofollow, nosnippet
Cache-Control: no-cache, no-store, max-age=0, must-revalidate
Pragma: no-cache
Expires: Mon, 01 Jan 1990 00:00:00 GMT
Date: Sat, 21 Dec 2024 09:42:17 GMT
P3P: CP="This is not a P3P policy! See g.co/p3phelp for more info."
Content-Security-Policy: require-trusted-types-for 'script';report-uri https://csp.withgoogle.com/csp/docs-tt
Referrer-Policy: origin
X-Content-Type-Options: nosniff
X-Frame-Options: SAMEORIGIN
X-XSS-Protection: 1; mode=block
Server: GSE
Set-Cookie: NID=520=qB3oU1nkXL1FlaUFKdsXLyFSNdQAYRgrxUQSIgkgufnqZ-v4gEw_AwJCs-AhV4RGdyITYQM6PuSo57lJ3otqT08GdaHLrO2ZqlneukuGYuUgKQFA96vRINv5Ki8PAWnA7WT_65Jjq37BtKIIukwGn7IwQvXidhKJsCiK4k2zJ0ENGdAi0g; expires=Sun, 22-Jun-2025 09:42:17 GMT; path=/; domain=.google.com; HttpOnly
Alt-Svc: h3=":443"; ma=2592000,h3-29=":443"; ma=2592000
Accept-Ranges: n

BadZipFile: File is not a zip file